<a href="https://colab.research.google.com/github/ttury/dcinside_corpus_croller/blob/main/dc_web_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---constant---

BASE_URL_REG = "https://gall.dcinside.com/board/lists" # 정식갤
ARTICLE_BASE_URL = "https://gall.dcinside.com" # 게시물
NORMAL_ARTICLE_URL_REG = "https://gall.dcinside.com/board/view/" # 공지 제외 게시물

BASE_URL_MINI = "https://gall.dcinside.com/mini/board/lists" # 미니갤
NORMAL_ARTICLE_URL_MINI = "https://gall.dcinside.com/mini/board/view/"

BASE_URL_MOBILE = "https://m.dcinside.com/board/maplestory/" # 모바일

GALARIES = {'야갤' : 'baseball_new10', '메갤' : 'maplestory', '대고갤' : 'eowjsrhemdgkrry', '식물갤' : 'tree'}

# -------------

# ---hyper parameter---

GALARY_ID = GALARIES['식물갤'] # 갤러리 ID
MAX_SLEEP_TIME = 3 # 크롤링 지연 시간 -> 차단 회피
BASE_URL = BASE_URL_REG
NORMAL_ARTICLE_URL = NORMAL_ARTICLE_URL_REG

# 헤더 설정, 차단을 피하기 위함
headers = {
    "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Encoding" : "gzip, deflate, br",
    "Accept-Language" : "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control" : "max-age=0",
    "Connection" : "keep-alive",
    "Host" : "gall.dcinside.com",
    "Referer" : "https://mc9w4ageo7e-496ff2e9c6d22116-0-colab.googleusercontent.com/",
    "sec-ch-ua" : 'Chromium";v="90", " Not A;Brand";v="99", "Whale";v="2"',
    "Sec-Fetch-Dest" : 'document',
    "sec-ch-ua-mobile" : "?0",
    "Sec-Fetch-Mode" : "navigate",
    "Sec-Fetch-Site" : "cross-site",
    "Sec-Fetch-User" : "?1",
    "Upgrade-Insecure-Requests" : "1",
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.229 Whale/2.10.123.42 Safari/537.36",
    }

# ---------------------

In [ ]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
from random import randint
import re
import pandas as pd
from urllib.error import HTTPError
from urllib.error import URLError

data = []
# 1페이지 ~ 10페이지
for i in range(1, 2):
  # 파라미터 설정(갤러리 설정)
  params = {'id': GALARY_ID,'page':i}

  # 갤러리 메인 접속
  try:
    response = requests.get(BASE_URL, params=params, headers=headers)
  except HTTPError as e:
    print('http error')
    continue
  except URLError as e:
    print('url error')
    continue
  soup = BeautifulSoup(response.content, 'html.parser')

  # 한 페이지에서 글 목록 긁어오기
  try:
    article_list = soup.find('tbody').find_all('tr')
  except AttributeError as e:
    print("Tag was not found")
    continue
  else:
    if soup.find('tbody') == None:
      print("Tag was not found")
      continue

  # 글 목록에서 각 게시물 별 접근
  for tr_item in article_list:

    # 각 게시물 제목, url 추출
    title_tag = tr_item.find('a', href=True)
    if title_tag == None:
      continue
    article_title = title_tag.text
    url = title_tag['href']
    article_url = ARTICLE_BASE_URL + url

    # 공지나 광고 같이 필요 없는 게시물은 무시
    if article_url.find(NORMAL_ARTICLE_URL) == -1:
      continue

    print('+'*12)
    print("제목: ", article_title)
    print("주소: ", article_url)
    print('+'*12)

    data.append([article_title, article_url, extract_corpus(article_title, article_url, params, headers)])
    random_time = randint(1, MAX_SLEEP_TIME)
    time.sleep(random_time)

columns = ['title', 'url', 'content']
dc_data = pd.DataFrame(data, columns=columns)

In [ ]:
print(dc_data[:5])
dc_data.to_csv('./drive/MyDrive/data/dc_data.csv')

In [ ]:
from urllib.request import urlopen, Request
from fake_useragent import UserAgent

useragent = UserAgent()
"""
headers = {
    'Host': 'gall.dcinside.com',
    'Connection': 'keep-alive',
    'Content-Length': '128',
    'sec-ch-ua': '"Chromium";v="90", " Not A;Brand";v="99", "Whale";v="2"',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.232 Whale/2.10.124.26 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://gall.dcinside.com',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://gall.dcinside.com/board/view/?id=dcbest&no=24366',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}
"""
headers = {
    'Host': 'gall.dcinside.com',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.232 Whale/2.10.124.26 Safari/537.36',
    'Referer': 'https://gall.dcinside.com/board/view/?id=dcbest&no=24366',
    'sec-ch-ua': '"Chromium";v="90", " Not A;Brand";v="99", "Whale";v="2"',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://gall.dcinside.com',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}
comment_url = 'https://gall.dcinside.com/board/comment/'
comment_response = requests.post(comment_url, headers=headers)

# print(comment_corpus[:3])

In [ ]:
pip install fake_useragent

In [ ]:
import json
json_data = comment_response.text
print(json_data)

In [ ]:
def extract_comment(article_soup):

  # 게시물에서 댓글 부분 긁어오기
  try:
    article_comments = article_soup.find('div', class_="comment_box")
  except AttributeError as e:
    print("Article Error")
    return
  else:
    if article_comments == None:
      print("Comments tag not found")
      return
  
  # 게시물 댓글에서 텍스트만 뽑아내기
  comments = []
  comment_group = []
  # comments 구조 : comments = [[comment, reply, reply], [comment], [comment, reply]]
  for article_comment in article_comments:
    # 댓글
    if article_comment.attrs['class'] == 'ub-content':
      comments.append(comment_group)
      comment = article_comment.find_all(p, class_='usertxt ub-word')
      comment_text = str(comment)
      comment_text = comment_text.replace('<br/>', '\n')
      comment_text = comment_text.replace('<br>', '\n')
      comment_text_soup = BeautifulSoup(comment_text)
      comment_text = comment_text_soup.get_text()
      comment_group = [comment_text]
    else:
      if article_comment.find(p, class_='usertxt ub-word') == Null:
        continue
      else:
        reply = article_comment.find(p, class_='usertxt ub-word')
        reply_text = str(reply)
        reply_text = reply_text.replace('<br/>', '\n')
        reply_text = reply_text.replace('<br>', '\n')
        reply_text_soup = BeautifulSoup(reply_text)
        reply_text = reply_text_soup.get_text()
        comment_group.append(reply_text)
  
  return comments
    

In [ ]:
def extract_corpus(article_title, article_url, params, headers):
    # 게시물 긁어오기
    try:
      article_response = requests.get(article_url, params=params, headers=headers)
    except HTTPError as e:
      print('http error')
      return
    except URLError as e:
      print('url error')
      return
    article_soup = BeautifulSoup(article_response.content, 'html.parser')
    
    article_corpus = extract_content(article_soup)
    return article_corpus


In [ ]:
def extract_content(article_soup):
  
    # 게시물에서 본문 부분 긁어오기
    try:
      article_contents = article_soup.find('div', class_="write_div")
    except AttributeError as e:
      print("Article Error")
      return
    else:
      if article_contents == None:
        print('Contents tag not found')
        return
    
    # dc 앱에서 작성한 글에 붙어 있는 헤더 삭제
    dcapp_tags = article_contents.find_all('span', id='dcappheader')
    for dcapp_tag in dcapp_tags:
      dcapp_tag.extract()
    
    # 게시물 본문에서 텍스트만 뽑아내기
    print("-------------")
    # 중복된 텍스트는 처리하지 않음
    previous_text = ''
    article_corpus = ''
    for article_content in article_contents:

      # <br> 태그를 줄바꿈으로 변환
      try:
        article_text = str(article_content)
        article_text = article_text.replace('<br/>', '\n')
        article_text = article_text.replace('<br>', '\n')
        article_text_soup = BeautifulSoup(article_text)
        article_text = article_text_soup.get_text()
      except:
        continue

      if article_text == previous_text:
        continue

      article_corpus += (article_text + "\n")
      previous_text = article_text

    article_corpus = re.sub(r'- dc official App', '', article_corpus) 
    article_corpus = re.sub(r'\n+', '\n', article_corpus)
    article_corpus = re.sub(r'^\n+', '', article_corpus)
    article_corpus = re.sub(r'\n+$', '', article_corpus)
    print(article_corpus)
    print("-------------\n")
    return article_corpus